## Exemplar Optimization via Genetic Search

We augment the original prompt optimization pipeline with an explicit exemplar optimization step, where the goal is to select a set of exemplars that maximizes task performance while controlling prompt length and redundancy among exemplars.

---

### Error-Weighted Exemplar Sampling with Temperature

To guide exemplar selection toward informative training examples while retaining exploration, we introduce an error-weighted sampling mechanism based on temperature-scaled softmax.

Let $c_i$ denote the accumulated error count for training example $e_i$. We define a sampling distribution over the training set as:

$$
P(e_i)
=
\frac{
\exp\left( \frac{c_i}{T} \right)
}{
\sum_j \exp\left( \frac{c_j}{T} \right)
}
$$

where \(T > 0\) is a temperature parameter controlling the entropy of the distribution.

---

### Effect of Temperature

- Large \(T\) produces a near-uniform distribution, encouraging exploration.
- Small \(T\) sharpens the distribution, concentrating probability mass on high-error examples.

In the limit:

$$
T \to \infty: \quad P(e_i) \approx \frac{1}{N}
$$

$$
T \to 0: \quad P(e_i) \text{ collapses to the highest-error example}
$$


This behavior mirrors temperature-scaled sampling in large language models, where temperature modulates randomness in token selection.

---

### Ojective Function

$$
\text{score}
=
m\!\left(p^*, e_1, \ldots, e_k\right)
-
\lambda_{\text{len}} \, k
-
\lambda_{\text{div}} \, R_{\text{div}}(E)
$$

$$
With: R_{\text{div}}(E)
=
\frac{1}{k(k-1)}
\sum_{i \ne j}
\operatorname{sim}(e_i, e_j)
$$


### Parameters

- $p^*$ : Optimized instruction prompt.
- $E = \{e_1, \ldots, e_k\}$ : Set of selected exemplars.
- $e_i$ : The $i$-th exemplar.
- $k = |E|$ : Number of exemplars in the prompt.

- $m(p^*, e_1, \ldots, e_k)$ :
  Task performance metric (e.g., accuracy, log-likelihood, reward)
  when using instruction $p^*$ together with exemplars $E$.

- $\lambda_{\mathrm{len}}$ :
  Length regularization coefficient penalizing large exemplar sets.

- $\lambda_{\mathrm{div}}$ :
  Diversity regularization coefficient.

- $\mathrm{sim}(e_i, e_j)$ :
  Similarity function between exemplars
  (e.g., cosine similarity of embeddings).

- $R_{\mathrm{div}}(E)$ :
  Average pairwise similarity among exemplars,
  encouraging diversity and reducing overfitting.


---
We optimize this objective using a genetic algorithm, where each individual represents a candidate exemplar set. Genetic operators such as selection, and mutation are applied to evolve exemplar populations toward higher objective scores. This formulation enables effective exploration of the combinatorial exemplar space while explicitly balancing performance, prompt length, and exemplar diversity.